In [28]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [29]:
class FinancialStatements: 
    
    def getelementinlist(list,element):
        try:
            return list[element]
        except:
            return '-'
    
    def toInt(financialStatements):
        indexes = financialStatements.index
        columns = financialStatements.columns
        for column in columns:
            for index in indexes:
                aux = financialStatements[column][index]
                if re.search(r'-',aux):
                    aux1 = float(re.sub(r'-','0',aux))
                    financialStatements[column][index] = int(aux1)
                if re.search(r'B',aux):
                    if re.search(r'[()]',aux):
                        aux1 = float(re.sub(r'[B()]','',aux))*1000000000*-1
                        financialStatements[column][index] = int(aux1)
                    else:
                        aux1 = float(re.sub(r'[B]','',aux))*1000000000
                        financialStatements[column][index] = int(aux1)
                if re.search(r'M',aux):
                    if re.search(r'[()]',aux):
                        aux1 = float(re.sub(r'[M()]','',aux))*1000000*-1
                        financialStatements[column][index] = int(aux1)
                    else:
                        aux1 = float(re.sub(r'[M]','',aux))*1000000
                        financialStatements[column][index] = int(aux1)
        return financialStatements
    
    def getFinancialStatements(self,Stocks, dates, root, folder, form):
        i = 0
        self.financial_statements = []
        self.stock_list = []
        no_of_stocks = len(Stocks.index)
        for i in range(no_of_stocks):
            try:
                urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials'
                urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials/balance-sheet'
                urlcashflow = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials/cash-flow'
                text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"lxml")
                text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"lxml")
                text_soup_cashflow = BeautifulSoup(requests.get(urlcashflow).text,"lxml")

                # Income Statement
                titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
                saleslist = []
                grossincomelist = []
                operatingincomelist = []
                pretaxincomelist = []
                netincomelist = []
                ebitdalist= []
                for title in titlesfinancials:
                    if 'Sales/Revenue' in title.text:
                        saleslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Gross Income' in title.text:
                        grossincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Operating Income' in title.text:
                        operatingincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Pretax Income' in title.text:
                        pretaxincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Income' in title.text:
                        netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'EBITDA' in title.text:
                        ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Balance sheet
                titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
                cashshortlist = []
                totalassetslist = []
                totalcurrentliabilitieslist = []
                totalliabilitieslist = [] 
                totalequitylist = []
                for title in titlesbalancesheet:
                    if 'Cash & Short Term Investments' in title.text:
                        cashshortlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Assets' in title.text:
                        totalassetslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Current Liabilities' in title.text:
                        totalcurrentliabilitieslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Liabilities' in title.text:
                        totalliabilitieslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Equity' in title.text:
                        totalequitylist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Cash Flow
                titlescashflow = text_soup_cashflow.findAll('td', {'class': 'rowTitle'})
                netoperatingcashflowlist = []
                netinvestingcashflowlist = []
                netfinincingcashflowlist = []
                freecashflowlist = []
                for title in titlescashflow:
                    if 'Net Operating Cash Flow' in title.text:
                        netoperatingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Investing Cash Flow' in title.text:
                        netinvestingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Financing Cash Flow' in title.text:
                        netfinincingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Free Cash Flow' in title.text:
                        freecashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Income Statement
                sales = FinancialStatements.getelementinlist(saleslist,0)
                grossincome = FinancialStatements.getelementinlist(grossincomelist,0)
                operatingincome = FinancialStatements.getelementinlist(operatingincomelist,0)
                pretaxincome = FinancialStatements.getelementinlist(pretaxincomelist,0)
                netincome = FinancialStatements.getelementinlist(netincomelist,0)
                ebitda = FinancialStatements.getelementinlist(ebitdalist,0)

                #Balance Sheet
                cashshort = FinancialStatements.getelementinlist(cashshortlist,0)
                totalassets = FinancialStatements.getelementinlist(totalassetslist,1)
                totalcurrentliabilities = FinancialStatements.getelementinlist(totalcurrentliabilitieslist,0)
                totalliabilities = FinancialStatements.getelementinlist(totalliabilitieslist,0)
                totalequity = FinancialStatements.getelementinlist(totalequitylist,0)

                #Cash Flow
                netoperatingcashflow = FinancialStatements.getelementinlist(netoperatingcashflowlist,0)
                netinvestingcashflow = FinancialStatements.getelementinlist(netinvestingcashflowlist,0)
                netfinancingcashflow = FinancialStatements.getelementinlist(netfinincingcashflowlist,0)
                freecashflow = FinancialStatements.getelementinlist(freecashflowlist,0)
                
                financialStatements= pd.DataFrame({'sales': sales,'grossincome': grossincome, 'ebitda': ebitda, 
                                                     'pretaxincome': pretaxincome,'netincome': netincome, 'cashshort': cashshort, 
                                                     'totalassets': totalassets,'totalcurrentliabilities': totalcurrentliabilities, 'totalliabilities': totalliabilities,
                                                     'totalequity': totalequity, 'netoperatingcashflow': netoperatingcashflow, 
                                                     'netinvestingcashflow': netinvestingcashflow, 'netfinancingcashflow': netfinancingcashflow,  
                                                     'freecashflow': freecashflow}, index = dates)
                self.financial_statements.append(FinancialStatements.toInt(financialStatements))
                self.stock_list.append(Stocks.iloc[i]['Symbols'])
#                 financial_statements.to_csv(root + folder + Stocks.iloc[i]['Symbols']+ 'FinancialStatements' + form, index = True, header=True)
            except:
                print('An error occurred on ' + Stocks.iloc[i]['Symbols'])

    def toCsv(self):
        i = 0
        for i in range(len(self.financial_statements)):
            self.financial_statements[i].to_csv(root + folder + self.stock_list[i] + 'FinancialStatements' + form, index = True, header=True)

In [30]:
dates = [2015,2016,2017,2018,2019]
root = './Stocks/'
folder = ''
form = '.csv'
Stocks = pd.read_csv('./Stocks.csv')

In [31]:
financialStatements = FinancialStatements()
financialStatements.getFinancialStatements(Stocks, dates, root, folder, form)

An error occurred on AIV
An error occurred on AMP
An error occurred on AMT
An error occurred on AON
An error occurred on AVB
An error occurred on AXP
An error occurred on BEN
An error occurred on BLK
An error occurred on CBOE
An error occurred on CME
An error occurred on DLR
An error occurred on DRE
An error occurred on EQR
An error occurred on ESS
An error occurred on ETFC
An error occurred on HST
An error occurred on ICE
An error occurred on IVZ
An error occurred on KIM
An error occurred on MAA
An error occurred on MMC
An error occurred on NDAQ
An error occurred on PEAK
An error occurred on PLD
An error occurred on REG
An error occurred on SBAC
An error occurred on SCHW
An error occurred on SLG
An error occurred on SPG
An error occurred on SYF
An error occurred on UDR
An error occurred on VNO
An error occurred on VTR
An error occurred on WELL
An error occurred on WLTW
An error occurred on WU
An error occurred on WY


In [32]:
financialStatements.toCsv()

In [38]:
financialStatements.stock_list[1]

'AAL'

In [39]:
financialStatements.financial_statements[1]

,sales,grossincome,ebitda,pretaxincome,netincome,cashshort,totalassets,totalcurrentliabilities,totalliabilities,totalequity,netoperatingcashflow,netinvestingcashflow,netfinancingcashflow,freecashflow
2015,40990000000,13980000000,8900000000,4620000000,7610000000,6950000000,48420000000,13610000000,42780000000,5640000000,6250000000,-5670000000,-1260000000,98000000
2016,40180000000,12660000000,7830000000,4300000000,2680000000,7000000000,51270000000,13870000000,47490000000,3790000000,6520000000,-5760000000,-894000000,793000000
2017,42620000000,12210000000,6980000000,3400000000,1280000000,5380000000,52790000000,15360000000,53570000000,-780000000,4740000000,-3640000000,-1150000000,-1230000000
2018,44540000000,11020000000,5610000000,1880000000,1410000000,4910000000,60580000000,18100000000,60750000000,-169000000,3530000000,-1970000000,-1670000000,-212000000
2019,45770000000,11420000000,6020000000,2260000000,1690000000,3980000000,60000000000,18310000000,60110000000,-118000000,3820000000,-2240000000,-1570000000,-453000000
